In [31]:
import pandas as pd
import sqlite3
import json

#cargaré los datos de los archivos JSON

import pandas as pd
import sqlite3
import json

# Cargar los datos desde archivos JSON
customers = pd.read_json("sample_analytics.customers.json")
accounts = pd.read_json("sample_analytics.accounts.json")

# Cargar y expandir el JSON de transacciones (está anidado)
with open("sample_analytics.transactions.json") as f:
    raw_transactions = json.load(f)

# Extraer y normalizar las transacciones por cuenta
transactions_expanded = []
for entry in raw_transactions:
    account_id = entry['account_id']  # id de cuenta que agrupa cada set de transacciones
    for tx in entry['transactions']:
        tx_flat = tx.copy()
        tx_flat['account_id'] = account_id
        if isinstance(tx['date'], dict):  # revisar si la fecha está en formato {'$date': ...}
            tx_flat['date'] = tx['date']['$date']
        transactions_expanded.append(tx_flat)

# Crear DataFrame de transacciones y limpiar
transactions = pd.DataFrame(transactions_expanded)
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')  # convertir a datetime
transactions = transactions[['account_id', 'date', 'amount', 'transaction_code', 'symbol', 'price', 'total']]

#normalizaremos los clientes
customers['birthdate'] = customers['birthdate'].apply(lambda x: x['$date'] if isinstance(x, dict) else x)
customers['birthdate'] = pd.to_datetime(customers['birthdate'], errors='coerce')
customers = customers.explode("accounts").rename(columns={'accounts': 'account_id'})  # una fila por cuenta
customers_final = customers[['username', 'name', 'birthdate', 'account_id']]

# Normalizar cuentas
accounts['products'] = accounts['products'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
accounts_final = accounts[['account_id', 'limit', 'products']]

# Crear base de datos SQLite y guardar las tablas
conn = sqlite3.connect("sample_analytics.db")
customers_final.to_sql("customers", conn, if_exists="replace", index=False)
accounts_final.to_sql("accounts", conn, if_exists="replace", index=False)
transactions.to_sql("transactions", conn, if_exists="replace", index=False)

88119

## Pregunta 1
¿Cuál es el promedio, mínimo, máximo y desviación estándar del límite de las cuentas de usuarios?

In [32]:
accounts_final["limit"].agg(['mean', 'min', 'max', 'std']).round(2)

,limit
mean,9955.90
min,3000.00
max,10000.00
std,354.75


## Pregunta 2
¿Cuántos clientes poseen más de una cuenta?

In [33]:
query = """SELECT COUNT(DISTINCT username) AS clientes_con_mas_de_una_cuenta
FROM (
    SELECT username, COUNT(DISTINCT account_id) AS cuentas
    FROM customers
    GROUP BY username
    HAVING cuentas > 1
);"""
pd.read_sql_query(query, conn)

,clientes_con_mas_de_una_cuenta
0,414


## Pregunta 3
¿Cuál es el monto promedio y el número de transacciones del mes de junio?

In [22]:
query = """SELECT
    ROUND(AVG(CAST(total AS FLOAT)), 2) AS monto_promedio,
    COUNT(*) AS cantidad_transacciones
FROM transactions
WHERE strftime('%m', date) = '06';"""
pd.read_sql_query(query, conn)

,monto_promedio,cantidad_transacciones
0,402121.61,7507


## Pregunta 4
¿Cuál es el id de cuenta con la mayor diferencia entre su transacción más alta y más baja?

In [23]:
query = """SELECT account_id,
       ROUND(MAX(CAST(total AS FLOAT)) - MIN(CAST(total AS FLOAT)), 2) AS diferencia
FROM transactions
GROUP BY account_id
ORDER BY diferencia DESC
LIMIT 1;"""
pd.read_sql_query(query, conn)

,account_id,diferencia
0,691668,8217481.36


## Pregunta 5
¿Cuántas cuentas tienen exactamente 3 productos y, además, uno de esos productos es "Commodity"?

In [24]:
query = """SELECT COUNT(*) AS cuentas_con_3_productos_y_commodity
FROM accounts
WHERE LENGTH(products) - LENGTH(REPLACE(products, ',', '')) = 2
  AND products LIKE '%Commodity%';"""
pd.read_sql_query(query, conn)

,cuentas_con_3_productos_y_commodity
0,202


## Pregunta 6
¿Cuál es el nombre del cliente que, en total entre todas sus cuentas, ha realizado la mayor cantidad de transacciones de tipo sell?

In [25]:
query = """SELECT c.name, SUM(t.amount) AS total_sell
FROM transactions t
JOIN customers c ON c.account_id = t.account_id
WHERE t.transaction_code = 'sell'
GROUP BY c.name
ORDER BY total_sell DESC
LIMIT 1;"""
pd.read_sql_query(query, conn)

,name,total_sell
0,John Williams,1155540


## Pregunta 7
¿Cuál es el usuario del cliente cuya cuenta tiene entre 10 y 20 transacciones de tipo “buy”, y que presenta el promedio de inversión más alto por operación de este tipo?

In [26]:
query = """SELECT c.username, ROUND(AVG(CAST(t.total AS FLOAT)), 2) AS promedio
FROM transactions t
JOIN customers c ON c.account_id = t.account_id
WHERE t.transaction_code = 'buy'
GROUP BY c.username
HAVING COUNT(*) BETWEEN 10 AND 20
ORDER BY promedio DESC
LIMIT 1;"""
pd.read_sql_query(query, conn)

,username,promedio
0,aspencer,1835289.71


## Pregunta 8
¿Cuál es el promedio de transacciones de compra y de venta por acción (campo “symbol”)?

In [34]:
query = """SELECT
  ROUND(AVG(cantidad_compras), 2) AS promedio_transacciones_compra_por_symbol
FROM (
  SELECT symbol, COUNT(*) AS cantidad_compras
  FROM transactions
  WHERE transaction_code = 'buy'
  GROUP BY symbol
);"""
pd.read_sql_query(query, conn)

,promedio_transacciones_compra_por_symbol
0,2448.83


In [35]:
query = """SELECT
  ROUND(AVG(cantidad_ventas), 2) AS promedio_transacciones_venta_por_symbol
FROM (
  SELECT symbol, COUNT(*) AS cantidad_ventas
  FROM transactions
  WHERE transaction_code = 'sell'
  GROUP BY symbol
);"""
pd.read_sql_query(query, conn)

,promedio_transacciones_venta_por_symbol
0,2446.67


## Pregunta 9
¿Cuáles son los diferentes benefi cios que tienen los clientes del tier “Gold”?

Después de revisar el campo tier_and_details, se encontró que:

No hay usuarios con tier = "Gold".
Por lo tanto, no existen beneficios asociados a usuarios con nivel "Gold" en este dataset.
Las siguientes celdas demuestran esta conclusión:

In [28]:
# Verificar cuántos usuarios tienen tier "Gold"
clientes_gold = customers[customers["tier_and_details"].apply(lambda x: x.get("tier") == "Gold")]
len(clientes_gold)

0

In [29]:
# Revisar contenido de beneficios en caso de que existiera alguno
clientes_gold["tier_and_details"].apply(lambda x: x.get("benefits", []))


,tier_and_details


## Pregunta 10
Obtener la cantidad de clientes por rangos etarios ([10–19], [20–29], etc.), que hayan realizado al menos una compra de acciones de “amzn”. La edad debe calcularse como la diferencia entre la fecha de corte 2025-05-16 y el campo “birthdate”.

In [30]:
query = """SELECT ((strftime('%Y', '2025-05-16') - strftime('%Y', birthdate)) / 10) * 10 AS rango_etario,
       COUNT(DISTINCT username) AS cantidad
FROM customers c
JOIN transactions t ON c.account_id = t.account_id
WHERE t.transaction_code = 'buy'
  AND t.symbol = 'amzn'
GROUP BY rango_etario
ORDER BY rango_etario;"""
pd.read_sql_query(query, conn)

,rango_etario,cantidad
0,NaN,18
1,20.0,9
2,30.0,52
3,40.0,48
4,50.0,24
